# 📊 OfficeMax México - Pronóstico de Ventas Back-to-School

## Análisis y Modelos Predictivos con Snowflake ML

Este notebook complementa el **Quickstart de OfficeMax** y demuestra cómo usar Machine Learning en SQL para crear modelos de pronóstico de ventas para la temporada escolar.

### Objetivos:
- 🎯 Crear modelos predictivos para productos Back-to-School
- 📈 Analizar tendencias de ventas por categoría
- 🔮 Generar pronósticos para optimización de inventario
- 💡 Proporcionar insights accionables para el negocio

### Datos utilizados:
- Productos de temporada escolar (Mochilas, Papelería, Tecnología, Arte)
- Ventas históricas de múltiples sucursales
- Datos de inventario y clientes
- Información de marketing y promociones

---
**Prerrequisitos**: Haber ejecutado el script `officemax-setup.sql` previamente.


In [ ]:
# Configuración inicial y conexión a Snowflake
import snowflake.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos con colores corporativos OfficeMax
plt.style.use('seaborn-v0_8')
officemax_colors = ['#003366', '#CC0000', '#0066CC', '#FF6600', '#669933', '#9966CC']

# Configuración de la conexión (ajustar según tu entorno)
conn_params = {
    'user': 'TU_USUARIO',
    'password': 'TU_PASSWORD',
    'account': 'TU_CUENTA.region',
    'warehouse': 'OFFICEMAX_WH',
    'database': 'OFFICEMAX_ANALYTICS',
    'schema': 'ANALYTICS'
}

# Establecer conexión
print("🔌 Conectando a Snowflake...")
try:
    conn = snowflake.connector.connect(**conn_params)
    print("✅ Conexión exitosa a Snowflake")
except Exception as e:
    print(f"❌ Error de conexión: {e}")
    print("📝 Recuerda ajustar los parámetros de conexión según tu entorno")


## 📊 Exploración de Datos

Comenzemos explorando los datos disponibles para entender los patrones de ventas de productos Back-to-School.


In [ ]:
# Cargar datos principales desde Snowflake
def ejecutar_query(query, descripcion=""):
    """Función helper para ejecutar queries y retornar DataFrames"""
    if descripcion:
        print(f"📋 {descripcion}")
    
    cursor = conn.cursor()
    try:
        df = pd.read_sql(query, conn)
        print(f"✅ Query ejecutada exitosamente - {len(df)} registros obtenidos")
        return df
    except Exception as e:
        print(f"❌ Error en query: {e}")
        return pd.DataFrame()
    finally:
        cursor.close()

# 1. Resumen de productos disponibles
query_productos = """
SELECT 
    CATEGORIA,
    SUB_CATEGORIA,
    COUNT(*) as TOTAL_PRODUCTOS,
    AVG(PRECIO_UNITARIO) as PRECIO_PROMEDIO,
    MIN(PRECIO_UNITARIO) as PRECIO_MIN,
    MAX(PRECIO_UNITARIO) as PRECIO_MAX
FROM RAW_DATA.PRODUCTOS 
WHERE TEMPORADA = 'Back-to-School'
GROUP BY CATEGORIA, SUB_CATEGORIA
ORDER BY CATEGORIA, PRECIO_PROMEDIO DESC
"""

df_productos = ejecutar_query(query_productos, "Cargando información de productos...")
print("\n📦 Resumen de Productos por Categoría:")
print(df_productos)
